<a href="https://colab.research.google.com/github/erikkemei/Kemei-Trial/blob/main/Fluoride_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
%matplotlib inline

In [3]:
df=pd.read_csv('https://raw.githubusercontent.com/erikkemei/Kemei-Trial/main/waterquality.csv')

UnicodeDecodeError: ignored

In [ ]:
df.head()

In [ ]:
df.drop(['Sample Time','Sample class','Sample Number','Sample Date','Sample Site','Location','Coliform (Quanti-Tray) (MPN /100mL)','E.coli(Quanti-Tray) (MPN/100mL)'],axis=1,inplace=True)

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.info()

**From this data, the total data is 50243, but the available data for fluoride is 6590. Alot of Fluoride data is apparaently missing**

In [ ]:
#Filling the missing values with 0
imputer=SimpleImputer(strategy='constant')
df2=pd.DataFrame(imputer.fit_transform(df),columns=df.columns)

In [ ]:
df2.describe()

In this section, the missing data has been filled with a zero value, by the impute function

In [ ]:
df2['Residual Free Chlorine (mg/L)'].replace('missing_value',0,inplace=True)
df2['Turbidity (NTU)'].replace('missing_value',0,inplace=True)
df2['Fluoride (mg/L)'].replace('missing_value',0,inplace=True)

In [ ]:
df2.describe()

In [ ]:
df3=df2[df2['Fluoride (mg/L)']>0]

In [ ]:
df3.describe()#information summary for al data without missing information

This is the filtered data on the basis of fluoride availability.

In [ ]:
df3.info()#Summary of the information on the data with with fluoride parameter put in consideration

In [ ]:
df3.head()

In [ ]:
from sklearn.manifold import TSNE
column_names=['Residual Free Chlorine (mg/L)','Turbidity (NTU)','Fluoride (mg/L)']
data_values=df3[column_names]
data_values_n=(data_values-data_values.min())/(data_values.max()-data_values.min());

In [ ]:
data_values_n

In [ ]:
tsne=TSNE(n_components=2, random_state=0)

In [ ]:
x_fld_2d=tsne.fit_transform(data_values_n)

In [ ]:
x_fld_2d

In [ ]:
df3['x']=x_fld_2d[:,0]
df3['y']=x_fld_2d[:,1]

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
**CLUSTERING OF THE DATA**
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [ ]:
k_rng=range(1,10)
sse=[]
for k in k_rng:
    km=KMeans(n_clusters=k)
    km.fit(df3[['Residual Free Chlorine (mg/L)','Turbidity (NTU)','Fluoride (mg/L)']])
    sse.append(km.inertia_)

In [ ]:
plt.xlabel('x')
plt.ylabel('sse')
plt.plot(k_rng,sse);
plt.title('GRAPH FOR DETERMINATION OF k');

From the Graph, the value of k is 2, which implies that the data can fit into two clusters

In [ ]:
km=KMeans(n_clusters=2)
y_predicted=km.fit_predict(df3[['Residual Free Chlorine (mg/L)','Turbidity (NTU)','Fluoride (mg/L)']])
df3['Cluster']=y_predicted

In [ ]:
df3.head(10)

In [ ]:
df4=df3[df3.Cluster==0]
df5=df3[df3.Cluster==1]

In [ ]:
sns.scatterplot(x='x',y='y', hue='Cluster',data=df3)
plt.xlabel('X-AXIS',fontsize=20)
plt.ylabel('Y-AXIS',fontsize=20)
plt.title('TSNE PLOT');

It is appaent that the data falls into two unique categories as shown

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
**DETERMINATION OF PERASON CORRELATION**
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [ ]:
correlation = df3.corr()

In [ ]:
print(correlation.loc['Residual Free Chlorine (mg/L)','Fluoride (mg/L)'])

In [ ]:
print(correlation.loc['Turbidity (NTU)','Fluoride (mg/L)'])

**SUMMARY: The correlation between Residual Free Chlorine and Fluoride is: -0.08 and that between turbidity and the Fluoride is : 0.18, both indicate a small level of association**

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
**PRINCIPAL COMPONENT ANALYSIS[PCA]**
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

In [ ]:
df3

In [ ]:
#Dropping the clolumns in bod1 for reuse in PCA analysis
df3.drop(['x','y','Cluster'],axis=1,inplace=True)
df4=df3

In [ ]:
df4

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(df4)
StandardScaler(copy=True,with_mean=True, with_std=True)
scaled_data=scaler.transform(df4)

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
pca.fit(scaled_data)
x_pca=pca.transform(scaled_data)

In [ ]:
#Re-Clustering of df4
km=KMeans(n_clusters=2)
y_predicted=km.fit_predict(df4[['Residual Free Chlorine (mg/L)','Turbidity (NTU)','Fluoride (mg/L)']])
df4['Cluster']=y_predicted

In [ ]:
per_var=np.round(pca.explained_variance_ratio_*100,decimals=1)
plt.figure(figsize=(10,8))
plt.scatter(x_pca[:,0],x_pca[:,1],c=df4['Cluster'])
plt.title('PRINCIPAL COMPONENT ANALYSIS[PCA] PLOT');
plt.xlabel('PC1-{0}%'.format(per_var[0]))
plt.ylabel('PC2-{0}%'.format(per_var[1]));

In [ ]:
plt.bar(x=range(1,len(per_var)+1),height=per_var)
plt.ylabel('% Explained Variance')
plt.xlabel('Pricipal Components')
plt.title('SCREE PLOT')
plt.show()

**As obtained from PCA plot too, the data seems to fall into two unique clusters, this is in agreement with TSNE too**